#### CS20M059 Shibobrota Das | CS20M007 Abhishek Kumar

## Setup

In [111]:
!pip install tensorflow-addons -qqq

In [112]:
!pip install wandb -qqq

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import GradientTape
from tensorflow import keras
import pandas as pd
import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, SimpleRNNCell, LSTMCell, GRUCell
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
import time
import sys
import datetime
from sklearn.utils import shuffle
import wandb
import csv
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("Using numpy:",np.__version__)
print("Using tensorflow:",tf.__version__)
print("Using tensorflow Addons:",tfa.__version__)
print("Using keras:",keras.__version__)
print("Using pandas:",pd.__version__)

Using numpy: 1.19.5
Using tensorflow: 2.5.0
Using tensorflow Addons: 0.13.0
Using keras: 2.5.0
Using pandas: 1.2.4


In [119]:
# wandb.init(project='Assignment 3', entity='iitm-cs6910-jan-may-2021-cs20m059-cs20m007')

In [12]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/DL-A3 Dataset/dakshina_dataset_v1.0/hi/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/DL-A3 Dataset/dakshina_dataset_v1.0/hi


#### Load Data

In [2]:
val_df = pd.read_csv("./lexicons/hi.translit.sampled.dev.tsv", sep='\t', header=None)
train_df = pd.read_csv("./lexicons/hi.translit.sampled.train.tsv", sep='\t', header=None)
test_df = pd.read_csv("./lexicons/hi.translit.sampled.test.tsv", sep='\t', header=None)
print("Data Loaded to Dataframes!")

Data Loaded to Dataframes!


In [14]:
%cd '/content/drive/My Drive/A3-checkpoints/'

/content/drive/My Drive/A3-checkpoints


#### Dataset Samples

In [4]:
train_df.sample(n=3)

,0,1,2
42980,स्वरूप,swarup,2
36244,वाल्मीक,valmik,2
42820,स्प्रिंकलर,sprinkler,3


## Preparing Dataset

In [5]:
sos = "@"
eos = "#"

In [6]:
class LexDataset:
    def __init__(self, input_tensor, target_tensor, batch_size):
        self.input_tensor = input_tensor
        self.target_tensor = target_tensor
        self.batch = tf.data.Dataset.from_tensor_slices((self.input_tensor, self.target_tensor)).shuffle(len(self.input_tensor)).batch(batch_size, drop_remainder=True)

In [7]:
class TransliterationDatatset:
    def __init__(self, df_list, batch_size = 64):
        
        self.input_tokenizer = None
        self.target_tokenizer = None
        self.train = None
        self.val = None
        self.test = None
        self.batch_size = batch_size
        # Load Data
        self.load_dataset(df_list)
        # Other parameters
        self.num_input_tokens = len(self.input_tokenizer.index_word)+1
        self.num_target_tokens = len(self.target_tokenizer.index_word)+1
        self.max_input_seq_length = np.max([self.train.input_tensor.shape[1], self.val.input_tensor.shape[1], self.test.input_tensor.shape[1]])
        self.max_target_seq_length = np.max([self.train.target_tensor.shape[1], self.val.target_tensor.shape[1], self.test.target_tensor.shape[1]])
        
    def preprocess_word(self, w):
        return sos + str(w) + eos
    
    def create_dataset(self, data_frame):
        input_words = []
        target_words = []
        # Shuffle the data_frame before creating dataset
        df = data_frame
        for i in range(5):
            df = shuffle(df)
        for x, y in zip(df[1], df[0]):
            input_words.append(str(x))
            target_words.append(self.preprocess_word(y))
        return (input_words, target_words)
    
    def load_dataset(self, df_list):
        # df_list should have train -> val -> test in sequence
        
        self.input_tokenizer = Tokenizer(num_words = None, char_level = True)
        self.target_tokenizer = Tokenizer(num_words = None, char_level = True)
        
        ds_list = []
        
        for df in df_list:
            # Get the words list
            (input_words, target_words) = self.create_dataset(df)
            # Fit on the set of words
            self.input_tokenizer.fit_on_texts(input_words)
            self.target_tokenizer.fit_on_texts(target_words)
            ds_list.append((input_words, target_words))
                    
        self.target_tokenizer.index_word.update({0:" "})
        self.input_tokenizer.index_word.update({0:" "})
        
        input_word_len = []
        target_word_len = []
        
        tensor_list = []
        
        for i, (input_words, target_words) in enumerate(ds_list):
            input_tensor = self.input_tokenizer.texts_to_sequences(input_words)
            target_tensor = self.target_tokenizer.texts_to_sequences(target_words)
            tensor_list.append((input_tensor, target_tensor))
            input_word_len.append(np.max([len(x) for x in input_tensor]))
            target_word_len.append(np.max([len(x) for x in target_tensor]))
        
        for i, (input_tensor, target_tensor) in enumerate(tensor_list):
            
            input_tensor = pad_sequences(input_tensor, padding='post', maxlen = np.max(input_word_len))
            target_tensor = pad_sequences(target_tensor, padding='post', maxlen = np.max(target_word_len))
            
            if i == 0:
                self.train = LexDataset(input_tensor, target_tensor, self.batch_size)
            elif i == 1:
                self.val = LexDataset(input_tensor, target_tensor, self.batch_size)
            else:
                self.test = LexDataset(input_tensor, target_tensor, self.batch_size)

In [8]:
dataset = TransliterationDatatset([train_df, val_df, test_df], 128)

#### Training Data

In [24]:
# Training data
dataset.train.input_tensor.shape, dataset.train.target_tensor.shape

((44204, 20), (44204, 21))

#### Validation Data

In [25]:
# Validation data
dataset.val.input_tensor.shape, dataset.val.target_tensor.shape

((4358, 20), (4358, 21))

#### Test Data

In [26]:
# Test data
dataset.test.input_tensor.shape, dataset.test.target_tensor.shape

((4502, 20), (4502, 21))

#### Number of Tokens

In [27]:
# Number of tokens
dataset.num_input_tokens, dataset.num_target_tokens

(28, 67)

#### Maximum Sequence Lengths

In [28]:
# max seq length
dataset.max_input_seq_length, dataset.max_target_seq_length

(20, 21)

#### Example batch - dataset

In [29]:
example_input_batch, example_target_batch = next(iter(dataset.train.batch))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 20]), TensorShape([128, 21]))

## Encoder and Decoder Model

In [30]:
latent_dim = 1024

In [31]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, dataset.num_input_tokens))
encoder_embedding = Embedding
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, dataset.num_target_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(dataset.num_target_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [32]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count

In [33]:
def word_accuracy(real, pred):
    
    pred = tf.cast(tf.argmax(pred, axis=2), tf.int32)
    real = tf.cast(tf.argmax(real, axis=2), tf.int32)
    correct_preds = tf.equal(pred, real)
    row_wise = tf.reduce_mean(tf.cast(correct_preds, tf.float32), axis=1)
    count = tf_count(row_wise, 1.0)
    
    return float(count)/float(len(row_wise))

In [34]:
model.compile(
    optimizer="nadam", loss="categorical_crossentropy", metrics=[word_accuracy]
)

In [35]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 28)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 67)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 1024), (None 4313088     input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 1024), 4472832     input_4[0][0]                    
                                                                 lstm_2[0][1]               

In [36]:
# Training
encoder_input_data = tf.one_hot(dataset.train.input_tensor, dataset.num_input_tokens)
decoder_input_data = tf.one_hot(dataset.train.target_tensor, dataset.num_target_tokens)

decoder_target_data = tf.one_hot(pad_sequences(dataset.train.target_tensor[:,1:], padding='post', maxlen = dataset.max_target_seq_length), 
                                 dataset.num_target_tokens)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=dataset.batch_size,
    epochs=1
)

 18/346 [>.............................] - ETA: 41:56 - loss: 1.9725 - word_accuracy: 0.0000e+00

KeyboardInterrupt: 